In [3]:
%matplotlib inline

from bs4 import BeautifulSoup
import matplotlib
import matplotlib.pyplot as plt
import re
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from webdriver_manager.chrome import ChromeDriverManager
import warnings

driver = webdriver.Chrome(ChromeDriverManager().install())
asset_dict = {} 

warnings.filterwarnings('ignore')


def get_table(soup):
    for t in soup.select('table'):
        header = t.select('thead tr th')
        if len(header) > 2:
            if (header[0].get_text().strip() == 'Symbol'
                and header[2].get_text().strip().startswith('% Holding')):
                return t
    raise Exception('could not find symbol list table')
    
# Scrapes ETF holdings from barchart.com
def get_etf_holdings():
    '''
    etf_symbol: str
    
    return: pd.DataFrame
    '''
    url = 'https://www.barchart.com/stocks/quotes/VONE/constituents?page=all'

    # Loads the ETF constituents page and reads the holdings table
    browser = WebDriver() # webdriver.PhantomJS()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html')
    table = get_table(soup)

    # Reads the holdings table line by line and appends each asset to a
    # dictionary along with the holdings percentage
    for row in table.select('tr')[1:26]:
        try:
            cells = row.select('td')
            # print(row)
            symbol = cells[0].get_text().strip()
            # print(symbol)
            # name = cells[1].text.strip()
            celltext = cells[2].get_text().strip()
            percent = float(celltext.rstrip('%'))
            # shares = int(cells[3].text.strip().replace(',', ''))
            if symbol != "" and percent != 0.0:
                asset_dict[symbol] = {
                    'percent': percent,
                }
        except BaseException as ex:
            print(ex)
    browser.quit()
    return pd.DataFrame(asset_dict)

constituent = get_etf_holdings()

constituent.T



Looking for [chromedriver 79.0.3945.36 win32] driver in cache
File found in cache by path [C:\Users\lukes\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]


,percent
AAPL,4.37
MSFT,3.99
AMZN,2.56
FB,1.64
BRK.B,1.48
JPM,1.45
GOOG,1.34
GOOGL,1.34
JNJ,1.30
V,1.08


{'AAPL': {'percent': 4.37}, 'MSFT': {'percent': 3.99}, 'AMZN': {'percent': 2.56}, 'FB': {'percent': 1.64}, 'BRK.B': {'percent': 1.48}, 'JPM': {'percent': 1.45}, 'GOOG': {'percent': 1.34}, 'GOOGL': {'percent': 1.34}, 'JNJ': {'percent': 1.3}, 'V': {'percent': 1.08}, 'PG': {'percent': 1.03}, 'XOM': {'percent': 0.99}, 'BAC': {'percent': 0.96}, 'T': {'percent': 0.95}, 'UNH': {'percent': 0.93}, 'MA': {'percent': 0.89}, 'INTC': {'percent': 0.87}, 'VZ': {'percent': 0.85}, 'DIS': {'percent': 0.84}, 'HD': {'percent': 0.81}, 'MRK': {'percent': 0.78}, 'CVX': {'percent': 0.77}, 'PFE': {'percent': 0.73}, 'KO': {'percent': 0.71}, 'CSCO': {'percent': 0.69}}


In [105]:
print(asset_dict)


{'AAPL': {'percent': 4.37}, 'MSFT': {'percent': 3.99}, 'AMZN': {'percent': 2.56}, 'FB': {'percent': 1.64}, 'BRK.B': {'percent': 1.48}, 'JPM': {'percent': 1.45}, 'GOOG': {'percent': 1.34}, 'GOOGL': {'percent': 1.34}, 'JNJ': {'percent': 1.3}, 'V': {'percent': 1.08}, 'PG': {'percent': 1.03}, 'XOM': {'percent': 0.99}, 'BAC': {'percent': 0.96}, 'T': {'percent': 0.95}, 'UNH': {'percent': 0.93}, 'MA': {'percent': 0.89}, 'INTC': {'percent': 0.87}, 'VZ': {'percent': 0.85}, 'DIS': {'percent': 0.84}, 'HD': {'percent': 0.81}, 'MRK': {'percent': 0.78}, 'CVX': {'percent': 0.77}, 'PFE': {'percent': 0.73}, 'KO': {'percent': 0.71}, 'CSCO': {'percent': 0.69}}
